In [1]:
import os
import json
import time
from tqdm import tqdm
import torch
from torch.profiler import profile, record_function, ProfilerActivity
import pandas as pd
from datasets import load_dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch.quantization
import torch.ao.quantization as tq
import gc

import jiwer
from jiwer import (
    Compose,
    ToLowerCase,
    RemoveMultipleSpaces,
    Strip,
)

# Final experiment
В этом ноутбуке проверяется гипотеза, что пропруненная ранее модель с базовой квантизацией и компиляцией даёт лучшие результаты. Прогон ноутбука произведён на тестовом сервере прунинга, поэтому результаты могут незначительно отличаться от метрик бэйзлайна.

Все значения профилировщика были сняты в прогоне на чистовую вне изолированной среды, поэтому значения могут отличаться, но статистически сводятся к итоговым метрикам.

In [2]:
def print_size_of_model(model):
    """ Prints the real size of the model """
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

In [3]:
def asr_metrics(hypothesis: str, reference: str):
    tr = Compose([ToLowerCase(), RemoveMultipleSpaces(), Strip()])

    ref_tr = tr(reference)
    hyp_tr = tr(hypothesis)

    out = jiwer.process_words(ref_tr, hyp_tr)
    wer = out.wer
    # S, D, I = out.substitutions, out.deletions, out.insertions

    cer = jiwer.cer(ref_tr, hyp_tr) # ?????

    return {
        "wer": wer,
        "cer": cer,
    }

In [4]:
def profile_sample(sample_idx=0, trace_path="whisper_perfetto_large-v3.json", sort_by="cpu_time_total", model=None):
    example = dataset[sample_idx]
    audio_array = example["audio"]["array"]
    sampling_rate = example["audio"]["sampling_rate"]

    inputs = processor(audio_array, sampling_rate=sampling_rate, return_tensors="pt").input_features

    with profile(
        activities=[ProfilerActivity.CPU],
        record_shapes=True,
        profile_memory=True,
        with_stack=False,
    ) as prof:
        with record_function("whisper.generate"):
            predicted_ids = model.generate(inputs, forced_decoder_ids=forced_decoder_ids)

    prof.export_chrome_trace(trace_path)
    print(f"Perfetto trace saved to {trace_path}")
    print(prof.key_averages().table(
        sort_by=sort_by,
        row_limit=10
    ))
    return processor.decode(predicted_ids[0])

In [5]:
dataset = load_dataset("bond005/sberdevices_golos_10h_crowd", split="validation", cache_dir="datasets")
# dataset = dataset.select(range(100))

In [6]:
processor = WhisperProcessor.from_pretrained("models--openai--whisper-large-v3/snapshots/06f233fe06e710322aca913c1bc4249a0d71fce1")
model = WhisperForConditionalGeneration.from_pretrained("models--openai--whisper-large-v3/snapshots/06f233fe06e710322aca913c1bc4249a0d71fce1")
forced_decoder_ids = processor.get_decoder_prompt_ids(language="russian", task="transcribe")
print_size_of_model(model)

Size (MB): 6174.372281


In [7]:
# Посмотрим на архитектуру модели
print(model)

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1280)
      (layers): ModuleList(
        (0-31): 32 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (fc2): Linear(in_features=5120, out_features=1280, bias=Tr

In [8]:
def run_model(verbose=False, model=None, dataset=None):
    results = []
    i = 0
    for audio in tqdm(dataset):
        audio_array = audio["audio"]["array"]
        sampling_rate = audio["audio"]["sampling_rate"]
        reference = audio["transcription"]
    
        start_time = time.time()
        input_features = processor(audio_array, sampling_rate=sampling_rate, return_tensors="pt").input_features 
        predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)[0] #Уточнить в зависимости от выбранной модели
        hypothesis = processor.decode(predicted_ids)
        run_time = time.time() - start_time
        metrics = asr_metrics(hypothesis, reference)
        metrics["run_time_sec"] = run_time
        if verbose:
            if i % 50 == 0:
                print("referenct:")
                print(reference)
                print("hypothesis:")
                print(hypothesis)
            i += 1
        results.append(metrics)

    df_results = pd.DataFrame(results)
    
    summary = {
        "total_samples": len(df_results),
        "avg_wer": df_results["wer"].mean(),
        "avg_cer": df_results["cer"].mean(),
        "avg_time_per_audio": df_results["run_time_sec"].mean(),
        "total_time": df_results["run_time_sec"].sum(),
    }
    
    print("large-v3")
    print(json.dumps(summary, ensure_ascii=True, indent=2))
    return summary

# CPU
Профилировка базовой модели

In [9]:
_ = profile_sample(116, trace_path="whisper_perfetto_large-v3_base.json", model=model)

Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Perfetto trace saved to whisper_perfetto_large-v3_base.json
-----------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                 Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
-----------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                     whisper.generate         3.99%     332.154ms       100.00%        8.325s        8.325s         112 B      -6.39 GB             1  
                                         aten::linear         0.48%      39.801ms        71.79%        5.976s       1.454ms       2.55 GB           0 B          4111  
                                          aten::addmm        57.63%        4.797s        59.86%     

In [11]:
# del model, _
# gc.collect()

1288802

Профилировка запруненной модели

In [10]:
model.load_state_dict(torch.load("whisper_pruned_iter3.pt"))

<All keys matched successfully>

In [12]:
_ = profile_sample(116, trace_path="whisper_perfetto_large-v3_pruned_base.json", model=model)

Perfetto trace saved to whisper_perfetto_large-v3_pruned_base.json
-----------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                 Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
-----------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                     whisper.generate         3.06%     236.690ms       100.00%        7.732s        7.732s         112 B      -6.39 GB             1  
                                         aten::linear         0.49%      38.039ms        74.94%        5.794s       1.409ms       2.55 GB           0 B          4111  
                                          aten::addmm        62.04%        4.796s        62.9

In [13]:
dataset_small = dataset.select(range(50))
_ = run_model(model=model, dataset=dataset_small)

100%|█████████████████████████████████████████████████████████████████| 50/50 [06:12<00:00,  7.46s/it]

large-v3
{
  "total_samples": 50,
  "avg_wer": 0.3959776334776335,
  "avg_cer": 0.1406573182624397,
  "avg_time_per_audio": 7.44771089553833,
  "total_time": 372.3855447769165
}


In [14]:
del dataset_small, _
gc.collect()

1288846

Явно видно полусекундное ускорение. Топ операций не поменялся, впрочем, это было ожидаемо.

# PTQ Dynamic
Простейшая восьмибитная квантизация в одну строчку.

In [15]:
print_size_of_model(model)

Size (MB): 6174.372281


In [16]:
modules_to_quantize = {torch.nn.Linear}
qmodel = tq.quantize_dynamic(
    model, 
    modules_to_quantize, 
    dtype=torch.qint8
)
print_size_of_model(qmodel)


/tmp/ipykernel_536/1967412630.py:2: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  qmodel = tq.quantize_dynamic(


Size (MB): 1837.108365


In [17]:
_ = profile_sample(116, trace_path="whisper_perfetto_large-v3_quanted_pruned.json", model=qmodel)

Perfetto trace saved to whisper_perfetto_large-v3_quanted_pruned.json
-----------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                 Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
-----------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                     whisper.generate         6.57%     377.013ms       100.00%        5.735s        5.735s         112 B      -6.39 GB             1  
                            quantized::linear_dynamic        61.47%        3.526s        62.10%        3.561s     866.279us       2.55 GB      -2.55 GB          4111  
                   aten::scaled_dot_product_attention         0.09%       5.023ms        1

Мы получили значительное ускорение инференса. Попробуем снять замеры качества с учётом torch.compile

In [19]:
qmodel = torch.compile(qmodel)
_ = profile_sample(116, trace_path="whisper_perfetto_large-v3_quanted_pruned_compiled.json", model=qmodel)

Perfetto trace saved to whisper_perfetto_large-v3_quanted_pruned_compiled.json
-----------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                 Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
-----------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                     whisper.generate         6.52%     360.600ms       100.00%        5.534s        5.534s         112 B      -6.39 GB             1  
                            quantized::linear_dynamic        61.67%        3.413s        62.35%        3.451s     839.412us       2.55 GB      -2.55 GB          4111  
                   aten::scaled_dot_product_attention         0.08%       4.604ms

In [20]:
summary = run_model(verbose=True, model=qmodel, dataset=dataset)

  0%|                                                               | 1/793 [00:05<1:13:54,  5.60s/it]

referenct:
можешь включить сериал теория большого взрыва
hypothesis:
 Можешь включить сериал «Теория большого взрыва»?


  6%|███▉                                                          | 51/793 [04:22<1:03:30,  5.14s/it]

referenct:
покажи на смотрешке канал бридж тв
hypothesis:
 Покажи на Смотрёжке канал Бридж ТВ.


 13%|████████                                                       | 101/793 [08:38<57:59,  5.03s/it]

...

referenct:
вячеслав владимирович месяцев
hypothesis:
 Вячеслав Владимирович Месяцев


 95%|█████████████████████████████████████████████████████████▊   | 751/793 [1:03:44<03:31,  5.03s/it]

referenct:
футбольный матч тоттенхэм лестер
hypothesis:
 Футбольный матч Тоттенхэм-Лестер.


100%|█████████████████████████████████████████████████████████████| 793/793 [1:07:18<00:00,  5.09s/it]

large-v3
{
  "total_samples": 793,
  "avg_wer": 0.4815027581357973,
  "avg_cer": 0.16642186304337375,
  "avg_time_per_audio": 5.083663366781959,
  "total_time": 4031.3450498580933
}


In [24]:
with open("whisper_metric.json", "r", encoding="utf-8") as f:
    data = json.load(f)

data["large-v3_cpu_quanted_pruned"] = summary

with open("whisper_metric.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=True, indent=2)

In [14]:
subdata = {
    k: data[k]
    for k in [
        "large-v3",
        "large-v3_cuda",
        "large-v3_cpu_quanted",
        "large-v3_cpu_global_magnitude_pruning_0.81",
        "large-v3_cpu_quanted_pruned",
    ]
}

In [15]:
pd.DataFrame(subdata)

,large-v3,large-v3_cuda,large-v3_cpu_quanted,large-v3_cpu_global_magnitude_pruning_0.81,large-v3_cpu_quanted_pruned
total_samples,793.000000,793.000000,793.000000,793.000000,793.000000
avg_wer,0.440303,0.440303,0.473953,0.440990,0.481503
avg_cer,0.158293,0.158293,0.166380,0.161862,0.166422
avg_time_per_audio,7.948655,0.838627,5.805705,7.437316,5.083663
total_time,6303.283459,665.031494,4603.923963,5897.791256,4031.345050


In [23]:
# Сохраняем state_dict квантованной модели
torch.save(qmodel.state_dict(), "./whisper-large-v3-quantized-dynamic-pruned.pth")

# Также сохраните конфигурацию отдельно (она не меняется)
qmodel.config.save_pretrained("./whisper-large-v3-quantized-dynamic-pruned")

/mnt/c/Users/Odmen/documents/venv/lib/python3.12/site-packages/transformers/configuration_utils.py:407: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 448, 'begin_suppress_tokens': [220, 50257]}
  warnings.warn(


In [52]:
del qmodel, _
gc.collect()

NameError: name 'qmodel' is not defined